<a href="https://colab.research.google.com/github/tiago-luisi/Mathematics-Statistics-DS-AI/blob/main/DSA_MATEST_Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='white'>Data Science Academy</font>
## <font color='white'>Matemática e Estatística Aplicada Para Data Science, Machine Learning e IA</font>
## <font color='white'>Projeto 1</font>
## <font color='white'>Vetores e Espaço Vetorial em Sistemas de Recomendação</font>

### Instalando e Carregando Pacotes

In [4]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:00


In [5]:
# Imports
import ast
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando e Compreendendo os Dados

In [7]:
# Dataset de filmes
df_dsa_filmes = pd.read_csv("dados/dataset_filmes.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'dados/dataset_filmes.csv'

In [ ]:
# Shape
df_dsa_filmes.shape

In [ ]:
# Amostra dos dados
df_dsa_filmes.head()

In [ ]:
# Dataset de elenco dos filmes
df_elenco = pd.read_csv("dados/dataset_elenco.csv")

In [ ]:
# Shape
df_elenco.shape

In [ ]:
# Amostra dos dados
df_elenco.head()

## Organização dos Dados de Texto

> Existe uma relação entre os 2 datasets. Vamos fazer o merge pela coluna comum entre eles, a coluna title.

In [ ]:
# Merge dos dataframes
df_dsa_filmes = df_dsa_filmes.merge(df_elenco, on = 'title')

In [ ]:
# Shape
df_dsa_filmes.shape

In [ ]:
# Amostra dos dados
df_dsa_filmes.head()

In [ ]:
# Info
df_dsa_filmes.info()

> Muitas colunas parecem não ser relevantes para nossa análise. Vamos manter apenas o que é relevante.

In [ ]:
# Filtrando o dataframe e mantendo somente as colunas consideradas mais relevantes
df_dsa_filmes = df_dsa_filmes[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

A decisão acima é sua, Cientista de Dados, com base no seu conhecimento sobre a área de negócio ou sua percepção sobre os dados. Tome a decisão, justique-a e siga em frente no processo de análise. Se mais tarde achou que não foi a melhor decisão, então volte, muda a decisão, justique novamente e siga em frente até chegar ao melhor resultado possível.

Lembre-se: Estamos fazendo Ciência de Dados e ciência requer experimentação!

In [ ]:
df_dsa_filmes.head(10)

## Análise Exploratória e Limpeza dos Dados

In [ ]:
# Shape
df_dsa_filmes.shape

In [ ]:
# Checando valores ausentes
df_dsa_filmes.isnull().sum()

In [ ]:
# Removemos as 3 linhas com valores ausentes
df_dsa_filmes.dropna(inplace = True)

In [ ]:
# Checando valores ausentes
df_dsa_filmes.isnull().sum()

In [ ]:
# Checando se há valores duplicados
df_dsa_filmes.duplicated().sum()

In [ ]:
# Shape
df_dsa_filmes.shape

## Processamento de Texto com Abstract Syntax Trees

https://docs.python.org/3/library/ast.html

Vamos começar ajustando a coluna de gênero de filme. Nesta coluna precisamos apenas dos nomes e não dos ids.

In [ ]:
df_dsa_filmes.head()

https://docs.python.org/3/library/ast.html

O módulo ast ajuda a descobrir programaticamente como é a gramática atual de um objeto, processando árvores da gramática de sintaxe abstrata.

In [ ]:
# Vejamos um exemplo
# Avaliamos a lista de dicionários de strings
ast.literal_eval('[{"id": 28, "name": "Action"}, \
                   {"id": 12, "name": "Adventure"}, \
                   {"id": 14, "name": "Fantasy"}, \
                   {"id": 878, "name": "Science Fiction"}]')

In [ ]:
# Função de conversão
# Este loop itera sobre os elementos de uma estrutura de dados.
# Usamos ast.literal_eval(obj) para avaliar/converter a string obj em uma estrutura de dados Python.
def converter(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
# Vamos testar a função
teste = converter('[{"id": 28, "name": "Action"}, \
                    {"id": 12, "name": "Adventure"}, \
                    {"id": 14, "name": "Fantasy"}, \
                    {"id": 878, "name": "Science Fiction"}]')

In [ ]:
print(teste)

In [ ]:
# Aplicamos então a função ao nosso dataset, na coluna de gênero do filme
df_dsa_filmes['genres'] = df_dsa_filmes['genres'].apply(converter)

In [ ]:
df_dsa_filmes.head()

In [ ]:
# Aplicamos então a função ao nosso dataset, na coluna de keywords do filme
df_dsa_filmes['keywords'] = df_dsa_filmes['keywords'].apply(converter)

In [ ]:
df_dsa_filmes.head()

Vamos aplicar o ast à coluna de cast (elenco) mas extrair sobre 3 membros do elenco. Isso deve ser suficiente.

In [ ]:
# Função de conversão
def converter3(obj):

    # Cria uma lista vazia chamada L para armazenar os nomes.
    L = []

    # Inicializa um contador para controlar o número de elementos adicionados à lista.
    counter = 0

    # Usa ast.literal_eval para avaliar a string 'obj' e iterar sobre os elementos resultantes.
    for i in ast.literal_eval(obj):

        # Verifica se o contador é diferente de 3.
        if counter != 3:

            # Adiciona o valor da chave 'name' do dicionário atual à lista L.
            L.append(i['name'])

            # Incrementa o contador.
            counter += 1
        else:

            # Interrompe o loop se o contador chegar a 3.
            break

    # Retorna a lista L com até 3 nomes.
    return L

In [ ]:
# Aplicamos então a função ao nosso dataset, na coluna de cast (elenco) do filme
df_dsa_filmes['cast'] = df_dsa_filmes['cast'].apply(converter3)

In [ ]:
df_dsa_filmes.tail()

A função abaixo vai extrair o nome do diretor do filme.

In [ ]:
# Função de conversão
def fetch_director(obj):

    # Cria uma lista vazia chamada L para armazenar o nome do diretor.
    L = []

    # Converte a string 'obj' em um objeto Python e itera sobre seus elementos.
    for i in ast.literal_eval(obj):

        # Verifica se o trabalho ('job') do elemento atual é 'Director'.
        if i['job'] == 'Director':

            # Adiciona o nome do diretor (valor da chave 'name') à lista L.
            L.append(i['name'])

            # Interrompe o loop após encontrar o primeiro diretor.
            break

    # Retorna a lista L, contendo o nome do diretor.
    return L

In [ ]:
# Aplicamos então a função ao nosso dataset, na coluna de crew (equipe) do filme
df_dsa_filmes['crew'] = df_dsa_filmes['crew'].apply(fetch_director)

In [ ]:
df_dsa_filmes.head()

## Limpeza dos Dados de Texto

In [ ]:
# Separamos a string da coluna overview por espaço em branco
df_dsa_filmes['overview'] = df_dsa_filmes['overview'].apply(lambda x:x.split())

In [ ]:
df_dsa_filmes.head()

In [ ]:
# Replace de espaço por vazio (remove o espaço)
df_dsa_filmes['genres'] = df_dsa_filmes['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
# Replace de espaço por vazio (remove o espaço)
df_dsa_filmes['keywords'] = df_dsa_filmes['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
# Replace de espaço por vazio (remove o espaço)
df_dsa_filmes['cast'] = df_dsa_filmes['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
# Replace de espaço por vazio (remove o espaço)
df_dsa_filmes['crew'] = df_dsa_filmes['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
df_dsa_filmes.head()

## Preparando os Dados Para Vetorização em Outro Espaço Vetorial

In [ ]:
# Criamos a coluna de tags, nesse caso um vetor de strings com os valores das colunas
df_dsa_filmes['tags'] = df_dsa_filmes['overview'] + \
                        df_dsa_filmes['genres'] + \
                        df_dsa_filmes['keywords'] + \
                        df_dsa_filmes['cast'] + \
                        df_dsa_filmes['crew']

In [ ]:
df_dsa_filmes.head()

In [ ]:
# Dataset final
df_dsa_filmes_novo = df_dsa_filmes[['movie_id', 'title', 'tags']]

In [ ]:
df_dsa_filmes_novo.head(10)

In [ ]:
# Join das strings para simplificar o vetor
df_dsa_filmes_novo['tags'] = df_dsa_filmes_novo['tags'].apply(lambda x:" ".join(x))

In [ ]:
# Coloca tudo em minúsculo para evitar diferença de palavras maiúsculo/minúsculo
df_dsa_filmes_novo['tags'] = df_dsa_filmes_novo['tags'].apply(lambda x:x.lower())

In [ ]:
df_dsa_filmes_novo.head(10)

## Parse e Vetorização

In [ ]:
# Criamos o parser
parser_ps = PorterStemmer()

Stemming é o processo de redução de uma palavra ao seu radical que está ligado a sufixos e prefixos ou às raízes de palavras conhecidas como "lemmas". Stemming é importante na compreensão de linguagem natural e processamento de linguagem natural.

In [ ]:
# Função de stemming
def stem(text):

    # Cria uma lista vazia chamada y para armazenar as palavras após o stemming.
    y = []

    # Divide a string de entrada 'text' em palavras e itera sobre elas.
    for i in text.split():

        # Realiza o stemming na palavra atual 'i' e adiciona o resultado à lista y.
        y.append(parser_ps.stem(i))

    # Retorna as palavras processadas como uma string, unindo-as com espaços.
    return " ".join(y)

In [ ]:
# Aplica a função à coluna de tags
df_dsa_filmes_novo['tags'] = df_dsa_filmes_novo['tags'].apply(stem)

In [ ]:
df_dsa_filmes_novo.head()

In [ ]:
# Cria o vetorizador com no máximo 5000 atributos
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

A vetorização no contexto de processamento de linguagem natural (PLN) é o processo de converter texto em uma representação numérica, geralmente na forma de vetores. Este processo é fundamental para que algoritmos de aprendizado de máquina possam trabalhar com dados textuais, uma vez que eles requerem entradas numéricas.

A linha de código acima está criando uma instância da classe CountVectorizer do scikit-learn. Vamos detalhar o que acontece nesta linha:

**CountVectorizer**: É uma técnica de vetorização que converte uma coleção de documentos de texto em uma matriz de contagens de tokens. Basicamente, ela conta quantas vezes cada palavra ocorre em cada documento.

**max_features = 5000**: Este parâmetro limita o número de palavras mais frequentes a serem consideradas. Aqui, apenas as 5000 palavras mais frequentes em todo o conjunto de documentos serão mantidas.

**stop_words = 'english'**: Especifica que as palavras de parada (stop words) em inglês devem ser removidas do texto. Palavras de parada são palavras que são filtradas antes ou após o processamento de texto, geralmente porque são muito comuns e carregam pouca informação significativa (por exemplo, "the", "is", "and" em inglês).

In [ ]:
# Cria os vetores para as tags
vectors = cv.fit_transform(df_dsa_filmes_novo['tags']).toarray()

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
type(vectors)

In [ ]:
vectors

In [ ]:
# Para visualizar todas as colunas do array
np.set_printoptions(threshold = np.inf)

A matriz **vectors** gerada pela função fit_transform do CountVectorizer representa a frequência de cada palavra (após a aplicação de stemming) em cada documento (ou linha) da coluna de tags de seu dataframe. Aqui está o que significa cada elemento dessa matriz:

0: Indica que a palavra correspondente a essa posição na matriz não aparece na linha específica do dataframe. Ou seja, a tag após o stemming não está presente naquele registro específico.

1: Significa que a palavra aparece uma vez na linha do dataframe. Após o stemming, a tag foi identificada uma vez naquele registro.

2 (ou mais): Indica que a palavra aparece múltiplas vezes. Um valor de 2 significa que, após o stemming, a tag específica foi encontrada duas vezes naquele registro, e assim por diante para valores maiores.

Este comportamento é esperado porque o CountVectorizer constrói um vocabulário de todas as palavras únicas encontradas nos documentos fornecidos e, em seguida, conta quantas vezes cada palavra aparece em cada documento. O processo de stemming aplicado anteriormente por meio da função stem reduz as palavras a suas raízes, o que pode resultar na agregação de diferentes formas da mesma palavra como uma única entrada no vocabulário. Isso pode levar a um aumento na contagem (por exemplo, "run", "running", "runs" após stemming podem ser contados como a mesma palavra base, aumentando sua contagem em um documento específico).

In [ ]:
vectors[0]

In [ ]:
vectors[1]

Os vetores acima ajudam a explicar porque é difícil treinar LLMs (Large Language Models), o estado da arte hoje em IA. Imagine vetores como esses para idiomas completos!

## Cálculo de Distância dos Vetores

A célula abaixo trabalha com os vetores no espaço de dimensão para calcular a distância matemática entre eles usando a distância de cosseno (cosine_similarity).

In [ ]:
# Calcula a similaridade entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors)

## Construindo o Sistema de Recomendação

In [ ]:
# Função para o sistema de recomendação
def sistema_recomendacao(movie):

    # Obtém o índice do filme passado como argumento (o que o usuário assistiu)
    index = df_dsa_filmes_novo[df_dsa_filmes_novo['title'] == movie].index[0]

    # Verificamos então os filmes com vetores de menor distância para o filme passado como argumento
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1])

    # E então consideramos os 5 filmes com menor distância, ou seja, maior similaridade
    for i in distances[1:6]:
        print(df_dsa_filmes_novo.iloc[i[0]].title)

## Aplicando o Sistema de Recomendação

In [ ]:
# Quais as recomendações de filmes para quem assistiu ao filme: Avengers: Age of Ultron?
sistema_recomendacao('Avengers: Age of Ultron')

In [ ]:
# Quais as recomendações de filmes para quem assistiu ao filme: Jurassic World?
sistema_recomendacao('Jurassic World')

In [ ]:
# Quais as recomendações de filmes para quem assistiu ao filme: The Hobbit: The Battle of the Five Armies?
sistema_recomendacao('The Hobbit: The Battle of the Five Armies')

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim